In [8]:
import numpy as np
import pandas as pd
import vtk
import os
from functions import export_to_vtp

np.random.seed(23765)

In [11]:
user = 'Kattelijn'
simulation = 'viz-disable'

if user == 'Kattelijn':
    base_folder = f'/Users/Kattelijn/Downloads/{simulation}'
elif user == 'Kevin':
    base_folder = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/{simulation}/'

### Load data

In [12]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv(os.path.join(base_folder, 'positions/rank_0_positions.txt'), skiprows= 8, delimiter=' ', names=column_headers_pos)

df_pos.head()

,local id,pos x,pos y,pos z,area,type
0,1,88.017654,143.88911,83.259131,area_43,ex
1,2,88.071787,143.79220,83.237149,area_43,ex
2,3,88.093434,143.95190,83.168331,area_43,ex
3,4,88.063898,143.81981,83.317563,area_43,ex
4,5,88.066131,143.97354,83.238470,area_43,ex


In [13]:
neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=100, replace=False))

In [14]:
column_headers_monitors = ['step', 'calcium']

df_calcium = pd.DataFrame()

for i in neurons:
    filename = f'monitors/0_{i}.csv'
    filepath = os.path.join(base_folder, filename)
    temp_df = pd.read_csv(filepath, skiprows=0, usecols=[0, 5], delimiter=';', names=column_headers_monitors)
    temp_df['neuron_id'] = i  # Add the neuron ID as a new column
    df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

df_calcium.tail()

,step,calcium,neuron_id
999995,999500,0.709484,49853
999996,999600,0.710379,49853
999997,999700,0.709282,49853
999998,999800,0.712179,49853
999999,999900,0.711064,49853


### Save data to VTP files

In [15]:
folder = "100neurons_vtp_files_disable"
time_steps = np.arange(0, 100000, 1000)

for time_point in time_steps:
    # Filter calcium data for the current time step
    df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

    # Merge with positions
    df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

    # Export to VTP file
    filename = f'neurons_timestep_{time_point}.vtp'
    export_to_vtp(df_merged, folder, filename)

    print(f"Saved: {os.path.join(folder, filename)}")

Saved: 100neurons_vtp_files_disable/neurons_timestep_0.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_1000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_2000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_3000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_4000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_5000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_6000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_7000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_8000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_9000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_10000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_11000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_12000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_13000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_14000.vtp
Saved: 100neurons_vtp_files_disable/neurons_timestep_15000.vtp
Saved